In [1]:
import pandas as pd
from datetime import datetime,date
import sqlalchemy as sq
import mysql.connector
from pandas.io import sql


# Data Cleaning and Data Wrangling using Pandas
## Import Raw Weather Datasets
Raw data is provided by Agriculture and Irrigation, Alberta Environment and Parks and Environment Canada. 

Due to the technical limitations of the site, only three years of data can be downloaded at a time. I, therefore, downloaded the data for 2018 to 2020 and 2021 in two parts and merged them into the original four-year weather data after processing using Pandas.

In [2]:
csv1 = pd.read_csv("ACISDailyData-20180101-20201231-PID143937394.csv", encoding= 'unicode_escape')
display(csv1)
csv1.nunique()

,Station Name,Date (Local Standard Time),Air Temp. Min. (°C),Air Temp. Min. Source Flag,Air Temp. Min. Record Completeness (%),Air Temp. Max. (°C),Air Temp. Max. Source Flag,Air Temp. Max. Record Completeness (%),Air Temp. Avg. (°C),Air Temp. Avg. Source Flag,Air Temp. Avg. Record Completeness (%),Relative Humidity Avg. (%),Relative Humidity Avg. Source Flag,Relative Humidity Avg. Record Completeness (%),Wind Speed 10 m Avg. (km/h),Wind Speed 10 m Avg. Source Flag,Wind Speed 10 m Avg. Record Completeness (%)
0,Calgary Int'L CS,01-January-2018,-32.2,ACTUAL,100.0,-3.6,ACTUAL,100.0,-18.7,AGGREGATED,100.0,70.2,AGGREGATED,100.0,6.8,AGGREGATED,100.0
1,Calgary Int'L CS,02-January-2018,-11.6,ACTUAL,100.0,0.8,ACTUAL,100.0,-5.8,AGGREGATED,100.0,66.9,AGGREGATED,100.0,5.9,AGGREGATED,100.0
2,Calgary Int'L CS,03-January-2018,-10.1,ACTUAL,100.0,6.4,ACTUAL,100.0,-3.7,AGGREGATED,100.0,71.2,AGGREGATED,100.0,4.5,AGGREGATED,100.0
3,Calgary Int'L CS,04-January-2018,-12.0,ACTUAL,100.0,4.0,ACTUAL,100.0,-6.4,AGGREGATED,100.0,80.3,AGGREGATED,100.0,4.2,AGGREGATED,100.0
4,Calgary Int'L CS,05-January-2018,-12.4,ACTUAL,100.0,9.6,ACTUAL,100.0,-1.6,AGGREGATED,100.0,70.6,AGGREGATED,100.0,7.3,AGGREGATED,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,Cop Upper,27-December-2020,-15.3,COMPUTED,0.0,-7.9,COMPUTED,0.0,-10.0,AGGREGATED,0.0,90.2,AGGREGATED,0.0,7.0,AGGREGATED,0.0
2188,Cop Upper,28-December-2020,-15.2,COMPUTED,4.2,-1.5,ACTUAL,4.2,-7.8,AGGREGATED,4.2,76.3,AGGREGATED,4.2,7.2,AGGREGATED,4.2
2189,Cop Upper,29-December-2020,-17.2,COMPUTED,0.0,-5.4,COMPUTED,0.0,-10.7,AGGREGATED,0.0,74.0,AGGREGATED,0.0,7.9,AGGREGATED,0.0
2190,Cop Upper,30-December-2020,-9.1,COMPUTED,0.0,-0.1,COMPUTED,0.0,-3.2,AGGREGATED,0.0,59.6,AGGREGATED,0.0,10.0,AGGREGATED,0.0


Station Name                                         2
Date (Local Standard Time)                        1096
Air Temp. Min. (°C)                                426
Air Temp. Min. Source Flag                           3
Air Temp. Min. Record Completeness (%)              17
Air Temp. Max. (°C)                                495
Air Temp. Max. Source Flag                           2
Air Temp. Max. Record Completeness (%)              16
Air Temp. Avg. (°C)                                451
Air Temp. Avg. Source Flag                           1
Air Temp. Avg. Record Completeness (%)              16
Relative Humidity Avg. (%)                         618
Relative Humidity Avg. Source Flag                   1
Relative Humidity Avg. Record Completeness (%)      15
Wind Speed 10 m Avg. (km/h)                        234
Wind Speed 10 m Avg. Source Flag                     1
Wind Speed 10 m Avg. Record Completeness (%)        17
dtype: int64

In [13]:
#display(csv1.loc[csv1['Date (Local Standard Time)'] == '31-December-2020'])
#Separating datasets by Station Name
df_cspre21 = csv1[csv1['Station Name'] == 'Calgary Int\'L CS']
df_cupre21 = csv1[csv1['Station Name'] == 'Cop Upper']
display(df_cspre21)
display(df_cupre21)

,Station Name,Date (Local Standard Time),Air Temp. Min. (°C),Air Temp. Min. Source Flag,Air Temp. Min. Record Completeness (%),Air Temp. Max. (°C),Air Temp. Max. Source Flag,Air Temp. Max. Record Completeness (%),Air Temp. Avg. (°C),Air Temp. Avg. Source Flag,Air Temp. Avg. Record Completeness (%),Relative Humidity Avg. (%),Relative Humidity Avg. Source Flag,Relative Humidity Avg. Record Completeness (%),Wind Speed 10 m Avg. (km/h),Wind Speed 10 m Avg. Source Flag,Wind Speed 10 m Avg. Record Completeness (%)
1095,Calgary Int'L CS,31-December-2020,-11.2,ACTUAL,100.0,0.3,ACTUAL,100.0,-5.3,AGGREGATED,100.0,76.0,AGGREGATED,100.0,5.5,AGGREGATED,100.0
2191,Cop Upper,31-December-2020,-9.9,COMPUTED,4.2,-1.2,ACTUAL,4.2,-4.9,AGGREGATED,4.2,74.7,AGGREGATED,4.2,7.0,AGGREGATED,4.2


,Station Name,Date (Local Standard Time),Air Temp. Min. (°C),Air Temp. Min. Source Flag,Air Temp. Min. Record Completeness (%),Air Temp. Max. (°C),Air Temp. Max. Source Flag,Air Temp. Max. Record Completeness (%),Air Temp. Avg. (°C),Air Temp. Avg. Source Flag,Air Temp. Avg. Record Completeness (%),Relative Humidity Avg. (%),Relative Humidity Avg. Source Flag,Relative Humidity Avg. Record Completeness (%),Wind Speed 10 m Avg. (km/h),Wind Speed 10 m Avg. Source Flag,Wind Speed 10 m Avg. Record Completeness (%)
0,Calgary Int'L CS,01-January-2018,-32.2,ACTUAL,100.0,-3.6,ACTUAL,100.0,-18.7,AGGREGATED,100.0,70.2,AGGREGATED,100.0,6.8,AGGREGATED,100.0
1,Calgary Int'L CS,02-January-2018,-11.6,ACTUAL,100.0,0.8,ACTUAL,100.0,-5.8,AGGREGATED,100.0,66.9,AGGREGATED,100.0,5.9,AGGREGATED,100.0
2,Calgary Int'L CS,03-January-2018,-10.1,ACTUAL,100.0,6.4,ACTUAL,100.0,-3.7,AGGREGATED,100.0,71.2,AGGREGATED,100.0,4.5,AGGREGATED,100.0
3,Calgary Int'L CS,04-January-2018,-12.0,ACTUAL,100.0,4.0,ACTUAL,100.0,-6.4,AGGREGATED,100.0,80.3,AGGREGATED,100.0,4.2,AGGREGATED,100.0
4,Calgary Int'L CS,05-January-2018,-12.4,ACTUAL,100.0,9.6,ACTUAL,100.0,-1.6,AGGREGATED,100.0,70.6,AGGREGATED,100.0,7.3,AGGREGATED,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,Calgary Int'L CS,27-December-2020,-12.6,ACTUAL,100.0,-7.7,ACTUAL,100.0,-8.9,AGGREGATED,100.0,90.6,AGGREGATED,100.0,6.8,AGGREGATED,100.0
1092,Calgary Int'L CS,28-December-2020,-14.2,ACTUAL,100.0,-3.1,ACTUAL,100.0,-8.0,AGGREGATED,100.0,78.6,AGGREGATED,100.0,5.7,AGGREGATED,100.0
1093,Calgary Int'L CS,29-December-2020,-16.4,ACTUAL,100.0,-5.8,ACTUAL,100.0,-10.8,AGGREGATED,100.0,75.0,AGGREGATED,100.0,4.7,AGGREGATED,100.0
1094,Calgary Int'L CS,30-December-2020,-10.9,ACTUAL,100.0,0.9,ACTUAL,100.0,-2.6,AGGREGATED,100.0,57.4,AGGREGATED,100.0,8.5,AGGREGATED,100.0


,Station Name,Date (Local Standard Time),Air Temp. Min. (°C),Air Temp. Min. Source Flag,Air Temp. Min. Record Completeness (%),Air Temp. Max. (°C),Air Temp. Max. Source Flag,Air Temp. Max. Record Completeness (%),Air Temp. Avg. (°C),Air Temp. Avg. Source Flag,Air Temp. Avg. Record Completeness (%),Relative Humidity Avg. (%),Relative Humidity Avg. Source Flag,Relative Humidity Avg. Record Completeness (%),Wind Speed 10 m Avg. (km/h),Wind Speed 10 m Avg. Source Flag,Wind Speed 10 m Avg. Record Completeness (%)
1096,Cop Upper,01-January-2018,-23.4,ACTUAL,100.0,-5.4,ACTUAL,100.0,-13.8,AGGREGATED,100.0,70.3,AGGREGATED,100.0,13.6,AGGREGATED,100.0
1097,Cop Upper,02-January-2018,-7.5,ACTUAL,100.0,0.7,ACTUAL,100.0,-2.5,AGGREGATED,100.0,59.4,AGGREGATED,100.0,13.4,AGGREGATED,100.0
1098,Cop Upper,03-January-2018,-2.2,ACTUAL,100.0,3.0,ACTUAL,100.0,-0.2,AGGREGATED,100.0,66.9,AGGREGATED,100.0,12.1,AGGREGATED,100.0
1099,Cop Upper,04-January-2018,-4.3,ACTUAL,100.0,1.1,ACTUAL,100.0,-1.8,AGGREGATED,100.0,70.0,AGGREGATED,100.0,8.5,AGGREGATED,100.0
1100,Cop Upper,05-January-2018,-3.1,ACTUAL,100.0,8.5,ACTUAL,100.0,2.9,AGGREGATED,100.0,63.8,AGGREGATED,100.0,11.8,AGGREGATED,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,Cop Upper,27-December-2020,-15.3,COMPUTED,0.0,-7.9,COMPUTED,0.0,-10.0,AGGREGATED,0.0,90.2,AGGREGATED,0.0,7.0,AGGREGATED,0.0
2188,Cop Upper,28-December-2020,-15.2,COMPUTED,4.2,-1.5,ACTUAL,4.2,-7.8,AGGREGATED,4.2,76.3,AGGREGATED,4.2,7.2,AGGREGATED,4.2
2189,Cop Upper,29-December-2020,-17.2,COMPUTED,0.0,-5.4,COMPUTED,0.0,-10.7,AGGREGATED,0.0,74.0,AGGREGATED,0.0,7.9,AGGREGATED,0.0
2190,Cop Upper,30-December-2020,-9.1,COMPUTED,0.0,-0.1,COMPUTED,0.0,-3.2,AGGREGATED,0.0,59.6,AGGREGATED,0.0,10.0,AGGREGATED,0.0


In [14]:
csv2 = pd.read_csv("ACISDailyData-20210101-20211231-PID144244925.csv", encoding= 'unicode_escape')
df_cs21 = csv2[csv2['Station Name'] == 'Calgary Int\'L CS']
df_cu21 = csv2[csv2['Station Name'] == 'Cop Upper']
display(df_cs21)
display(df_cu21)

,Station Name,Date (Local Standard Time),Air Temp. Min. (°C),Air Temp. Min. Source Flag,Air Temp. Min. Record Completeness (%),Air Temp. Max. (°C),Air Temp. Max. Source Flag,Air Temp. Max. Record Completeness (%),Air Temp. Avg. (°C),Air Temp. Avg. Source Flag,Air Temp. Avg. Record Completeness (%),Relative Humidity Avg. (%),Relative Humidity Avg. Source Flag,Relative Humidity Avg. Record Completeness (%),Wind Speed 10 m Avg. (km/h),Wind Speed 10 m Avg. Source Flag,Wind Speed 10 m Avg. Record Completeness (%)
0,Calgary Int'L CS,01-January-2021,-5.0,ACTUAL,100.0,3.2,ACTUAL,100.0,-0.3,AGGREGATED,100.0,58.8,AGGREGATED,100.0,10.9,AGGREGATED,100.0
1,Calgary Int'L CS,02-January-2021,-7.6,ACTUAL,100.0,7.4,ACTUAL,100.0,2.3,AGGREGATED,100.0,52.6,AGGREGATED,100.0,17.8,AGGREGATED,100.0
2,Calgary Int'L CS,03-January-2021,-6.5,ACTUAL,100.0,4.8,ACTUAL,100.0,1.6,AGGREGATED,100.0,52.3,AGGREGATED,100.0,23.8,AGGREGATED,100.0
3,Calgary Int'L CS,04-January-2021,-7.5,ACTUAL,100.0,2.1,ACTUAL,100.0,-1.4,AGGREGATED,100.0,59.7,AGGREGATED,100.0,12.3,AGGREGATED,100.0
4,Calgary Int'L CS,05-January-2021,-5.5,ACTUAL,100.0,4.9,ACTUAL,100.0,-0.2,AGGREGATED,100.0,62.6,AGGREGATED,100.0,15.6,AGGREGATED,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,Calgary Int'L CS,27-December-2021,-34.4,ACTUAL,100.0,-27.9,ACTUAL,100.0,-31.1,AGGREGATED,100.0,71.6,AGGREGATED,100.0,8.3,AGGREGATED,100.0
361,Calgary Int'L CS,28-December-2021,-32.4,ACTUAL,100.0,-15.8,ACTUAL,100.0,-23.0,AGGREGATED,100.0,73.8,AGGREGATED,100.0,12.6,AGGREGATED,100.0
362,Calgary Int'L CS,29-December-2021,-28.7,ACTUAL,100.0,-21.2,ACTUAL,100.0,-23.8,AGGREGATED,100.0,65.6,AGGREGATED,100.0,12.8,AGGREGATED,100.0
363,Calgary Int'L CS,30-December-2021,-23.2,ACTUAL,100.0,-16.3,ACTUAL,100.0,-19.2,AGGREGATED,100.0,74.0,AGGREGATED,100.0,11.5,AGGREGATED,100.0


,Station Name,Date (Local Standard Time),Air Temp. Min. (°C),Air Temp. Min. Source Flag,Air Temp. Min. Record Completeness (%),Air Temp. Max. (°C),Air Temp. Max. Source Flag,Air Temp. Max. Record Completeness (%),Air Temp. Avg. (°C),Air Temp. Avg. Source Flag,Air Temp. Avg. Record Completeness (%),Relative Humidity Avg. (%),Relative Humidity Avg. Source Flag,Relative Humidity Avg. Record Completeness (%),Wind Speed 10 m Avg. (km/h),Wind Speed 10 m Avg. Source Flag,Wind Speed 10 m Avg. Record Completeness (%)
365,Cop Upper,01-January-2021,-4.6,COMPUTED,0.0,2.6,COMPUTED,0.0,-0.6,AGGREGATED,0.0,60.2,AGGREGATED,0.0,12.8,AGGREGATED,0.0
366,Cop Upper,02-January-2021,-4.8,COMPUTED,0.0,7.2,COMPUTED,0.0,2.7,AGGREGATED,0.0,50.8,AGGREGATED,0.0,21.0,AGGREGATED,4.2
367,Cop Upper,03-January-2021,-3.9,COMPUTED,4.2,4.0,COMPUTED,4.2,1.2,AGGREGATED,4.2,53.2,AGGREGATED,4.2,25.3,AGGREGATED,4.2
368,Cop Upper,04-January-2021,-7.2,COMPUTED,0.0,1.5,COMPUTED,0.0,-2.0,AGGREGATED,0.0,58.9,AGGREGATED,0.0,13.7,AGGREGATED,0.0
369,Cop Upper,05-January-2021,-5.2,COMPUTED,0.0,4.0,COMPUTED,0.0,-0.8,AGGREGATED,0.0,65.1,AGGREGATED,0.0,16.5,AGGREGATED,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,Cop Upper,27-December-2021,-33.6,ACTUAL,100.0,-23.7,ACTUAL,100.0,-30.3,AGGREGATED,100.0,72.5,AGGREGATED,100.0,4.9,AGGREGATED,100.0
726,Cop Upper,28-December-2021,-24.6,ACTUAL,100.0,-15.9,ACTUAL,100.0,-20.7,AGGREGATED,100.0,73.9,AGGREGATED,100.0,9.8,AGGREGATED,100.0
727,Cop Upper,29-December-2021,-29.1,ACTUAL,100.0,-20.9,ACTUAL,100.0,-24.3,AGGREGATED,100.0,68.7,AGGREGATED,100.0,9.5,AGGREGATED,100.0
728,Cop Upper,30-December-2021,-23.4,ACTUAL,100.0,-16.1,ACTUAL,100.0,-18.8,AGGREGATED,100.0,75.5,AGGREGATED,100.0,7.0,AGGREGATED,100.0


In [17]:
df_cs = pd.concat([df_cspre21, df_cspre21])
display(df_cs)
df_cu = pd.concat([df_cupre21, df_cu21])
display(df_cu)
df_weather = pd.concat([df_cs, df_cu])
display(df_weather)


,Station Name,Date (Local Standard Time),Air Temp. Min. (°C),Air Temp. Min. Source Flag,Air Temp. Min. Record Completeness (%),Air Temp. Max. (°C),Air Temp. Max. Source Flag,Air Temp. Max. Record Completeness (%),Air Temp. Avg. (°C),Air Temp. Avg. Source Flag,Air Temp. Avg. Record Completeness (%),Relative Humidity Avg. (%),Relative Humidity Avg. Source Flag,Relative Humidity Avg. Record Completeness (%),Wind Speed 10 m Avg. (km/h),Wind Speed 10 m Avg. Source Flag,Wind Speed 10 m Avg. Record Completeness (%)
0,Calgary Int'L CS,01-January-2018,-32.2,ACTUAL,100.0,-3.6,ACTUAL,100.0,-18.7,AGGREGATED,100.0,70.2,AGGREGATED,100.0,6.8,AGGREGATED,100.0
1,Calgary Int'L CS,02-January-2018,-11.6,ACTUAL,100.0,0.8,ACTUAL,100.0,-5.8,AGGREGATED,100.0,66.9,AGGREGATED,100.0,5.9,AGGREGATED,100.0
2,Calgary Int'L CS,03-January-2018,-10.1,ACTUAL,100.0,6.4,ACTUAL,100.0,-3.7,AGGREGATED,100.0,71.2,AGGREGATED,100.0,4.5,AGGREGATED,100.0
3,Calgary Int'L CS,04-January-2018,-12.0,ACTUAL,100.0,4.0,ACTUAL,100.0,-6.4,AGGREGATED,100.0,80.3,AGGREGATED,100.0,4.2,AGGREGATED,100.0
4,Calgary Int'L CS,05-January-2018,-12.4,ACTUAL,100.0,9.6,ACTUAL,100.0,-1.6,AGGREGATED,100.0,70.6,AGGREGATED,100.0,7.3,AGGREGATED,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,Cop Upper,27-December-2021,-33.6,ACTUAL,100.0,-23.7,ACTUAL,100.0,-30.3,AGGREGATED,100.0,72.5,AGGREGATED,100.0,4.9,AGGREGATED,100.0
726,Cop Upper,28-December-2021,-24.6,ACTUAL,100.0,-15.9,ACTUAL,100.0,-20.7,AGGREGATED,100.0,73.9,AGGREGATED,100.0,9.8,AGGREGATED,100.0
727,Cop Upper,29-December-2021,-29.1,ACTUAL,100.0,-20.9,ACTUAL,100.0,-24.3,AGGREGATED,100.0,68.7,AGGREGATED,100.0,9.5,AGGREGATED,100.0
728,Cop Upper,30-December-2021,-23.4,ACTUAL,100.0,-16.1,ACTUAL,100.0,-18.8,AGGREGATED,100.0,75.5,AGGREGATED,100.0,7.0,AGGREGATED,100.0


,Station Name,Date (Local Standard Time),Air Temp. Min. (°C),Air Temp. Min. Source Flag,Air Temp. Min. Record Completeness (%),Air Temp. Max. (°C),Air Temp. Max. Source Flag,Air Temp. Max. Record Completeness (%),Air Temp. Avg. (°C),Air Temp. Avg. Source Flag,Air Temp. Avg. Record Completeness (%),Relative Humidity Avg. (%),Relative Humidity Avg. Source Flag,Relative Humidity Avg. Record Completeness (%),Wind Speed 10 m Avg. (km/h),Wind Speed 10 m Avg. Source Flag,Wind Speed 10 m Avg. Record Completeness (%)
1096,Cop Upper,01-January-2018,-23.4,ACTUAL,100.0,-5.4,ACTUAL,100.0,-13.8,AGGREGATED,100.0,70.3,AGGREGATED,100.0,13.6,AGGREGATED,100.0
1097,Cop Upper,02-January-2018,-7.5,ACTUAL,100.0,0.7,ACTUAL,100.0,-2.5,AGGREGATED,100.0,59.4,AGGREGATED,100.0,13.4,AGGREGATED,100.0
1098,Cop Upper,03-January-2018,-2.2,ACTUAL,100.0,3.0,ACTUAL,100.0,-0.2,AGGREGATED,100.0,66.9,AGGREGATED,100.0,12.1,AGGREGATED,100.0
1099,Cop Upper,04-January-2018,-4.3,ACTUAL,100.0,1.1,ACTUAL,100.0,-1.8,AGGREGATED,100.0,70.0,AGGREGATED,100.0,8.5,AGGREGATED,100.0
1100,Cop Upper,05-January-2018,-3.1,ACTUAL,100.0,8.5,ACTUAL,100.0,2.9,AGGREGATED,100.0,63.8,AGGREGATED,100.0,11.8,AGGREGATED,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,Cop Upper,27-December-2021,-33.6,ACTUAL,100.0,-23.7,ACTUAL,100.0,-30.3,AGGREGATED,100.0,72.5,AGGREGATED,100.0,4.9,AGGREGATED,100.0
726,Cop Upper,28-December-2021,-24.6,ACTUAL,100.0,-15.9,ACTUAL,100.0,-20.7,AGGREGATED,100.0,73.9,AGGREGATED,100.0,9.8,AGGREGATED,100.0
727,Cop Upper,29-December-2021,-29.1,ACTUAL,100.0,-20.9,ACTUAL,100.0,-24.3,AGGREGATED,100.0,68.7,AGGREGATED,100.0,9.5,AGGREGATED,100.0
728,Cop Upper,30-December-2021,-23.4,ACTUAL,100.0,-16.1,ACTUAL,100.0,-18.8,AGGREGATED,100.0,75.5,AGGREGATED,100.0,7.0,AGGREGATED,100.0


,Station Name,Date (Local Standard Time),Air Temp. Min. (°C),Air Temp. Min. Source Flag,Air Temp. Min. Record Completeness (%),Air Temp. Max. (°C),Air Temp. Max. Source Flag,Air Temp. Max. Record Completeness (%),Air Temp. Avg. (°C),Air Temp. Avg. Source Flag,Air Temp. Avg. Record Completeness (%),Relative Humidity Avg. (%),Relative Humidity Avg. Source Flag,Relative Humidity Avg. Record Completeness (%),Wind Speed 10 m Avg. (km/h),Wind Speed 10 m Avg. Source Flag,Wind Speed 10 m Avg. Record Completeness (%)
0,Calgary Int'L CS,01-January-2018,-32.2,ACTUAL,100.0,-3.6,ACTUAL,100.0,-18.7,AGGREGATED,100.0,70.2,AGGREGATED,100.0,6.8,AGGREGATED,100.0
1,Calgary Int'L CS,02-January-2018,-11.6,ACTUAL,100.0,0.8,ACTUAL,100.0,-5.8,AGGREGATED,100.0,66.9,AGGREGATED,100.0,5.9,AGGREGATED,100.0
2,Calgary Int'L CS,03-January-2018,-10.1,ACTUAL,100.0,6.4,ACTUAL,100.0,-3.7,AGGREGATED,100.0,71.2,AGGREGATED,100.0,4.5,AGGREGATED,100.0
3,Calgary Int'L CS,04-January-2018,-12.0,ACTUAL,100.0,4.0,ACTUAL,100.0,-6.4,AGGREGATED,100.0,80.3,AGGREGATED,100.0,4.2,AGGREGATED,100.0
4,Calgary Int'L CS,05-January-2018,-12.4,ACTUAL,100.0,9.6,ACTUAL,100.0,-1.6,AGGREGATED,100.0,70.6,AGGREGATED,100.0,7.3,AGGREGATED,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,Cop Upper,27-December-2021,-33.6,ACTUAL,100.0,-23.7,ACTUAL,100.0,-30.3,AGGREGATED,100.0,72.5,AGGREGATED,100.0,4.9,AGGREGATED,100.0
726,Cop Upper,28-December-2021,-24.6,ACTUAL,100.0,-15.9,ACTUAL,100.0,-20.7,AGGREGATED,100.0,73.9,AGGREGATED,100.0,9.8,AGGREGATED,100.0
727,Cop Upper,29-December-2021,-29.1,ACTUAL,100.0,-20.9,ACTUAL,100.0,-24.3,AGGREGATED,100.0,68.7,AGGREGATED,100.0,9.5,AGGREGATED,100.0
728,Cop Upper,30-December-2021,-23.4,ACTUAL,100.0,-16.1,ACTUAL,100.0,-18.8,AGGREGATED,100.0,75.5,AGGREGATED,100.0,7.0,AGGREGATED,100.0
